In [1]:
# | default_exp ctx

# CTX
> Utils for working with MRO CTX data

In [2]:
# | hide
from nbdev.showdoc import show_doc

In [3]:
# | export

import warnings
from itertools import repeat
from pathlib import Path

import hvplot.xarray  # noqa
import rasterio
import rioxarray as rxr
from fastcore.script import call_parse
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import process_map
from yarl import URL

from planetarypy.config import config
from planetarypy.pds.apps import get_index
from planetarypy.utils import catch_isis_error, file_variations, url_retrieve

try:
    from kalasiris.pysis import (
        cam2map,
        ctxcal,
        ctxevenodd,
        mroctx2isis,
        spiceinit,
    )
except KeyError:
    warnings.warn("kalasiris has a problem initializing ISIS")

In [4]:
# | export

warnings.filterwarnings("ignore", category=rasterio.errors.NotGeoreferencedWarning)
baseurl = URL(config.get_value("mro.ctx.datalevels.edr.url"))

In [5]:
# | export
storage_root = config.storage_root / "missions/mro/ctx"
cache = dict()

In [6]:
import pandas as pd

pd.set_option("display.max_columns", 1000)

In [7]:
# | export
def get_edr_index(refresh=False):
    if "edrindex" in cache and not refresh:
        return cache["edrindex"]
    else:
        edrindex = get_index("mro.ctx", "edr", refresh=refresh)
        edrindex["short_pid"] = edrindex.PRODUCT_ID.map(lambda x: x[:15])
        edrindex["month_col"] = edrindex.PRODUCT_ID.map(lambda x: x[:3])
        edrindex.LINE_SAMPLES = edrindex.LINE_SAMPLES.astype(int)
        cache["edrindex"] = edrindex
        return edrindex

In [8]:
get_edr_index(False)

,VOLUME_ID,FILE_SPECIFICATION_NAME,ORIGINAL_PRODUCT_ID,PRODUCT_ID,IMAGE_TIME,INSTRUMENT_ID,INSTRUMENT_MODE_ID,LINE_SAMPLES,LINES,SPATIAL_SUMMING,SCALED_PIXEL_WIDTH,PIXEL_ASPECT_RATIO,EMISSION_ANGLE,INCIDENCE_ANGLE,PHASE_ANGLE,CENTER_LONGITUDE,CENTER_LATITUDE,UPPER_LEFT_LONGITUDE,UPPER_LEFT_LATITUDE,UPPER_RIGHT_LONGITUDE,UPPER_RIGHT_LATITUDE,LOWER_LEFT_LONGITUDE,LOWER_LEFT_LATITUDE,LOWER_RIGHT_LONGITUDE,LOWER_RIGHT_LATITUDE,MISSION_PHASE_NAME,TARGET_NAME,SPACECRAFT_CLOCK_START_COUNT,FOCAL_PLANE_TEMPERATURE,LINE_EXPOSURE_DURATION,OFFSET_MODE_ID,SAMPLE_FIRST_PIXEL,SCALED_IMAGE_WIDTH,SCALED_IMAGE_HEIGHT,SPACECRAFT_ALTITUDE,TARGET_CENTER_DISTANCE,SLANT_DISTANCE,USAGE_NOTE,NORTH_AZIMUTH,SUB_SOLAR_AZIMUTH,SUB_SOLAR_LONGITUDE,SUB_SOLAR_LATITUDE,SUB_SPACECRAFT_LONGITUDE,SUB_SPACECRAFT_LATITUDE,SOLAR_DISTANCE,SOLAR_LONGITUDE,LOCAL_TIME,IMAGE_SKEW_ANGLE,RATIONALE_DESC,DATA_QUALITY_DESC,ORBIT_NUMBER,short_pid,month_col
0,MROX_0001,DATA/CRU_000001_9999_XN_99N999W.IMG,4A_04_0001000400,CRU_000001_9999_XN_99N999W,2005-08-30 15:40:21.549,CTX,NIFL,5056,1024,1,0.0,0.0,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,CRUISE,SPACE,0809883639:076,283.3,10.0,194/53/53,0,0.0,0.0,0.0,0.0,0.0,N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,278.89,10.16,0.0,Instrument checkout image of space ...,OK,-4242,CRU_000001_9999,CRU
1,MROX_0001,DATA/CRU_000002_9999_XN_99N999W.IMG,4A_04_0001000500,CRU_000002_9999_XN_99N999W,2005-09-08 15:59:45.313,CTX,NIFL,5056,15360,1,0.0,0.0,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,CRUISE,MOON,0810662403:012,296.0,5.71,196/243/238,0,0.0,0.0,0.0,0.0,0.0,N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,284.48,4.6,0.0,Calibration image of the Moon ...,OK,-4126,CRU_000002_9999,CRU
2,MROX_0001,DATA/CRU_000003_9999_XN_99N999W.IMG,4A_04_0001000600,CRU_000003_9999_XN_99N999W,2005-09-08 16:03:37.927,CTX,NIFL,5056,2048,1,0.0,0.0,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,CRUISE,STAR,0810662635:169,296.6,22.9,196/243/238,0,0.0,0.0,0.0,0.0,0.0,N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,284.48,4.66,0.0,Calibration image of Omega Centauri (globular ...,OK,-4126,CRU_000003_9999,CRU
3,MROX_0001,DATA/CRU_000004_9999_XN_99N999W.IMG,4A_04_0001000700,CRU_000004_9999_XN_99N999W,2005-09-08 16:08:23.841,CTX,NIFL,5056,2048,1,0.0,0.0,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,CRUISE,STAR,0810662921:147,296.8,22.9,196/243/238,0,0.0,0.0,0.0,0.0,0.0,N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,284.48,4.74,0.0,Calibration image of Omega Centauri (globular ...,OK,-4126,CRU_000004_9999,CRU
4,MROX_0001,DATA/CRU_000005_9999_XN_99N999W.IMG,4A_04_0001000800,CRU_000005_9999_XN_99N999W,2005-09-08 16:11:18.649,CTX,NIFL,5056,21504,1,0.0,0.0,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,999.9,CRUISE,MOON,0810663096:098,297.1,5.71,196/243/238,0,0.0,0.0,0.0,0.0,0.0,N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,284.48,4.79,0.0,Calibration image of the Moon ...,OK,-4126,CRU_000005_9999,CRU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149377,MROX_4896,DATA/V06_082476_1989_XI_18N075W.IMG,4A_04_11A3017300,V06_082476_1989_XI_18N075W,2024-02-29 20:23:08.773,CTX,ITL,5056,26624,1,5.71,1.05,7.84,64.15,56.71,74.95,18.97,75.03,17.6,74.53,17.66,75.37,20.28,74.85,20.34,ESP,MARS,1393705448:224,293.0,1.877,197/202/197,0,28.58,159.7,280.33,3674.45,282.77,N,276.78,162.84,132.16,-11.78,75.58,18.91,211484456.2,208.3,15.81,90.0,Sacra Sulci ...,OK,82476,V06_082476_1989,V06
149378,MROX_4896,DATA/V06_082477_1528_XI_27S096W.IMG,4A_04_11A3017400,V06_082477_1528_XI_27S096W,2024-02-29 22:01:18.785,CTX,ITL,5056,12288,1,5.35,1.14,12.61,58.01,45.62,96.13,-27.23,96.29,-27.89,95.79,-27.83,96.47,-26.64,95.97,-26.58,ESP,MARS,1393711338:227,292.8,1.877,197/202/197,0,26.79,74.7,255.43,3647.45,261.28,N,277.34,190.26,155.98,-11.8,97.13,-27.34,211475111.8,208.3

In [9]:
# | export
class CTXEDR:
    """Manage access to EDR data"""

    root = config.get_value("mro.ctx.datalevels.edr.root") or storage_root / "edr"
    with_pid_folder = config.get_value("mro.ctx.datalevels.edr.with_pid_folder")
    with_volume = config.get_value("mro.ctx.datalevels.edr.with_volume")

    def __init__(
        self,
        pid: str,  # CTX product id (pid)
        root: str = "",  # alternative root folder for EDR data
        with_volume=None,  # does the storage path include the volume folder
        with_pid_folder=None,  # control if stuff is stored inside PID folders
        refresh_index=False,
    ):
        self.pid = pid
        self.root = Path(root) if root else Path(self.root)
        self.with_volume = with_volume if with_volume is not None else self.with_volume
        self.with_pid_folder = (
            with_pid_folder if with_pid_folder is not None else self.with_pid_folder
        )
        self.refresh_index = refresh_index
        self.edrindex = None

    @property
    def pid(self):
        "Return product_id"
        return self._pid

    @pid.setter
    def pid(self, value):
        if len(value) < 26:
            val = value[:15]  # use short_pid
            self.edrindex = get_edr_index()
            value = self.edrindex.query(f"short_pid=='{val}'").PRODUCT_ID.iloc[0]
        self._pid = value

    @property
    def short_pid(self):
        return self.pid[:15]

    @property
    def meta(self):
        "get the metadata from the index table"
        edrindex = get_edr_index(refresh=self.refresh_index)
        s = edrindex.query("PRODUCT_ID == @self.pid").squeeze()
        s.index = s.index.str.lower()
        return s

    @property
    def volume(self):
        "get the PDS volume number for the current product id"
        return self.meta.volume_id.lower()

    @property
    def source_folder(self):
        """Calculate the source folder based on storage options `with_pid_folder` and `with_volume`."""
        base = self.root
        if self.with_volume:
            base = self.root / self.volume
        if self.with_pid_folder:
            base = base / self.pid
        return base

    @property
    def source_path(self):
        """Combine `source_folder` with `pid` into full path."""
        return self.source_folder / f"{self.pid}.IMG"

    @property
    def url(self):
        "Calculate URL from input dataframe row."
        url = baseurl / self.meta.volume_id.lower() / "data" / (self.pid + ".IMG")
        return url

    def download(self, overwrite=False):  # use `overwrite` to download in all cases.
        "Download and store correctly the EDR data, if not locally available."
        if self.source_path.exists() and not overwrite:
            print("File exists. Use `overwrite=True` to download fresh.")
            return
        self.source_folder.mkdir(parents=True, exist_ok=True)
        url_retrieve(self.url, self.source_path)

    def __str__(self):
        "Show some info about yourself when returned in a REPL (like ipython/jupyter)."
        s = f"PRODUCT_ID: {self.pid}\n"
        s += f"URL: {self.url}\n"
        s += f"source_path: {self.source_path}\n"
        return s

    def __repr__(self):
        return self.__str__()

PRODUCT_IDs can be provided in the shortened form (still unique), which are the first 15 characters of the full PRODUCT_ID:

In [10]:
pid = "F10_039666_1383"
pid = "B01_009958_1524_XI_27S347W"

In [11]:
edr = CTXEDR(pid)

In [12]:
show_doc(CTXEDR.pid)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L87){target="_blank" style="float:right; font-size:smaller"}

### CTXEDR.pid

>      CTXEDR.pid ()

*Return product_id*

In [13]:
edr.pid

'B01_009958_1524_XI_27S347W'

In [14]:
edr.short_pid

'B01_009958_1524'

These are the storage configuration settings:

In [15]:
edr.root

Path('/Users/maye/planetarypy_data/missions/mro/ctx/edr')

In [16]:
edr.with_pid_folder

''

In [17]:
edr.with_volume

''

In [18]:
show_doc(CTXEDR.source_folder)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L112){target="_blank" style="float:right; font-size:smaller"}

### CTXEDR.source_folder

>      CTXEDR.source_folder ()

*Calculate the source folder based on storage options `with_pid_folder` and `with_volume`.*

In [19]:
edr.source_folder

Path('/Users/maye/planetarypy_data/missions/mro/ctx/edr')

In [20]:
show_doc(CTXEDR.source_path)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L122){target="_blank" style="float:right; font-size:smaller"}

### CTXEDR.source_path

>      CTXEDR.source_path ()

*Combine `source_folder` with `pid` into full path.*

In [21]:
edr.source_path

Path('/Users/maye/planetarypy_data/missions/mro/ctx/edr/B01_009958_1524_XI_27S347W.IMG')

In [22]:
edr.source_path.exists()

False

In [23]:
show_doc(CTXEDR.meta)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L99){target="_blank" style="float:right; font-size:smaller"}

### CTXEDR.meta

>      CTXEDR.meta ()

*get the metadata from the index table*

In [24]:
edr.meta

volume_id                                                               MROX_0684
file_specification_name                       DATA/B01_009958_1524_XI_27S347W.IMG
original_product_id                                              4A_04_103100F800
product_id                                             B01_009958_1524_XI_27S347W
image_time                                             2008-09-10 10:15:05.533000
instrument_id                                                              CTX   
instrument_mode_id                                                          ITL  
line_samples                                                                 5056
lines                                                                       18432
spatial_summing                                                                 1
scaled_pixel_width                                                           5.13
pixel_aspect_ratio                                                           1.18
emission_angle  

In [25]:
show_doc(CTXEDR.url)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L127){target="_blank" style="float:right; font-size:smaller"}

### CTXEDR.url

>      CTXEDR.url ()

*Calculate URL from input dataframe row.*

In [26]:
show_doc(CTXEDR.download)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L132){target="_blank" style="float:right; font-size:smaller"}

### CTXEDR.download

>      CTXEDR.download (overwrite=False)

*Download and store correctly the EDR data, if not locally available.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| overwrite | bool | False | use `overwrite` to download in all cases. |

In [27]:
edr.download()

B01_009958_1524_XI_27S347W.IMG:   0%|          | 0/93197248 [00:00<?, ?it/s]

In [28]:
edr.source_path

Path('/Users/maye/planetarypy_data/missions/mro/ctx/edr/B01_009958_1524_XI_27S347W.IMG')

In [29]:
show_doc(CTXEDR.__str__)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L140){target="_blank" style="float:right; font-size:smaller"}

### CTXEDR.__str__

>      CTXEDR.__str__ ()

*Show some info about yourself when returned in a REPL (like ipython/jupyter).*

In [30]:
Path(config.get_value("mro.ctx.preproc_root"))

Path('.')

In [35]:
# | export
class CTX:
    """Class to manage dealing with CTX data.

    HAS a CTXEDR attribute as defined above.
    Attributes from CTXEDR are availalbe via __getattr__()
    """

    proc_root = storage_root / "edr"
    preproc_root = Path(config.get_value("mro.ctx.preproc_root"))
    preproc_calib_extension = config.get_value("mro.ctx.calib_extension")
    preproc_with_pid_folder = config.get_value("mro.ctx.preproc_with_pid_folder")
    preproc_with_volume = config.get_value("mro.ctx.preproc_with_volume")
    proc_with_pid_folder = config.get_value("mro.ctx.proc_with_pid_folder")
    proc_with_volume = config.get_value("mro.ctx.proc_with_volume")

    def __init__(
        self,
        id_: str,  # CTX product id
        source_dir: str = "",  # where the raw EDR data is stored, if not coming from plpy
        proc_root: str = "",  # where to store processed, if not plpy
        with_volume=None,  # store with extra volume subfolder?
        with_pid_folder=None,  # store with extra product_id subfolder?
        use_preproc=False,  # use preproc for cal_da
    ):
        self.edr = CTXEDR(id_, root=source_dir, with_volume=with_volume)
        self.proc_root = Path(proc_root) if proc_root else self.proc_root
        self.with_volume = with_volume if with_volume else self.proc_with_volume
        self.with_pid_folder = (
            with_pid_folder if with_pid_folder else self.proc_with_pid_folder
        )
        self.use_preproc = use_preproc

        (self.cub_name, self.cal_name, self.destripe_name, self.map_name) = (
            file_variations(
                self.edr.source_path.name,
                [".cub", ".cal.cub", ".dst.cal.cub", ".lev2.cub"],
            )
        )

        # status flags for caching
        self.is_read = False
        self.is_calib_read = False
        self.checked_destripe = False

    def __getattr__(self, attr):
        return getattr(self.edr, attr)

    @property
    def proc_folder(self) -> Path:
        "the folder for foreign processed data, like pre-processed calibrated data, e.g."
        path = self.proc_root
        if self.proc_with_volume:
            path = path / self.volume
        if self.proc_with_pid_folder:
            path = path / self.pid
        return path

    @property
    def preproc_folder(self) -> Path:
        "the folder for foreign processed data, like pre-processed calibrated data, e.g."
        path = self.preproc_root
        if self.preproc_with_volume:
            path = path / self.volume
        if self.preproc_with_pid_folder:
            path = path / self.pid
        return path

    @property
    def cub_path(self) -> Path:
        "Path to cube after import to ISIS."
        return self.proc_folder / self.cub_name

    @property
    def cal_path(self) -> Path:
        "Path to calibrated cube file. Also destriped files get this name."
        return self.proc_folder / self.cal_name

    @property
    def preproc_cal_path(self) -> Path:
        "Path to a preprocessend calibrated file"
        cal_name = file_variations(
            self.edr.source_path.name, [self.preproc_calib_extension]
        )[0]
        return self.preproc_folder / cal_name

    @property
    def destripe_path(self) -> Path:
        "One can keep destriped cubes as extra files, but it increases path management complexity."
        return self.proc_folder / self.destripe_name

    @property
    def map_path(self) -> Path:
        return self.proc_folder / self.map_name

    @catch_isis_error
    def isis_import(self) -> None:
        "Import EDR data into ISIS cube."
        self.cub_path.parent.mkdir(exist_ok=True, parents=True)
        mroctx2isis(from_=self.source_path, to=self.cub_path)

    @catch_isis_error
    def spice_init(self, web="yes") -> None:
        "Perform `spiceinit.`"
        spiceinit(from_=self.cub_path, web=web)

    @catch_isis_error
    def calibrate(self) -> None:
        "Do ISIS `ctxcal`."
        ctxcal(from_=self.cub_path, to=self.cal_path)
        self.is_calib_read = False

    @catch_isis_error
    def destripe(self, do_rename=True) -> None:
        "Do destriping via `ctxevenodd` if allowed by summing status."
        if self.spatial_summing != 2:
            ctxevenodd(from_=self.cal_path, to=self.destripe_path)
            if do_rename:
                self.destripe_path.rename(self.cal_path)

    @catch_isis_error
    def map_project(self, mpp=6.25) -> None:
        "Perform map projection."
        cam2map(from_=self.cal_path, to=self.map_path, pixres="mpp", resolution=mpp)

    @property
    def spatial_summing(self) -> int:
        "Get the spatial summing value from the index file."
        return self.meta["spatial_summing"]

    @property
    def data_quality(self) -> str:
        "Return the index file content for the DATA_QUALITY_DESC flag."
        return self.meta.data_quality_desc

    def calib_pipeline(self, overwrite=False) -> None:
        "Execute the whole ISIS pipeline for CTX EDR data."
        if self.cal_path.exists() and not overwrite:
            return
        pbar = tqdm("isis_import spice_init calibrate destripe".split())
        for name in pbar:
            pbar.set_description(name)
            getattr(self, name)()
        pbar.set_description("Done.")

    @property
    def edr_da(self):
        """Read EDR into xr.DataArray. Drop superfluous band dimension.

        If it was read before, use stored object for speed-up.
        'da' stands for data-array.
        """
        if not self.is_read:
            if not self.source_path.exists():
                # Doing this by hand because rasterio doesn't throw exception when path is missing.
                raise FileNotFoundError("EDR not downloaded yet.")
            self._edr_da = rxr.open_rasterio(self.source_path).sel(band=1, drop=True)
            self._edr_da.name = f"{self.short_pid} EDR"
            self.is_read = True
        return self._edr_da.drop_vars("spatial_ref")

    @property
    def edr_shape(self):
        return self.edr_da.shape

    @property
    def cal_da(self):
        """Read calibrated ISIS cube into xarray.DataArray using rioxarray.

        Drop superfluous `band` dimension.
        If it was read before, use stored object for speed-up.
        'da' stands for data-array.
        """
        if not self.is_calib_read:
            path = self.cal_path if not self.use_preproc else self.preproc_cal_path
            self._cal_da = rxr.open_rasterio(path, masked=True).sel(band=1, drop=True)
            self._cal_da.name = f"{self.short_pid} calibrated"
            self.is_calibd_read = True
        return self._cal_da.drop_vars("spatial_ref")

    @property
    def cal_shape(self):
        return self.cal_da.shape

    def plot_da(self, data):
        """Use hvplot to plot the xarray. Used by plot_calibrated to plot the calibrated array."""
        return data.hvplot(
            x="y",
            y="x",
            rasterize=True,
            cmap="gray",
            width=1000,
            height=400,
            title=self.pid[:15],
        )

    def plot_edr(self):
        "Plot EDR xarray using hvplot."
        return self.plot_da(self.edr_da)

    def plot_calibrated(self):
        "Plot the calibrated xarray using hvplot."
        return self.plot_da(self.cal_da)

    @property
    def tif_path(self):
        return self.proc_folder / self.map_name.with_suffix(".tif")

    def save_as_tif(self, refresh=False):
        if self.tif_path.is_file() and not refresh:
            print("File exists. Use `refresh=True` to force recreation.")
            return

        with rasterio.open(str(self.map_path)) as src:
            # Copy the metadata from the source file
            kwargs = src.meta.copy()
            # Update the data type if necessary
            kwargs.update(
                driver="GTiff",
                dtype=rasterio.float32,  # or whatever data type is appropriate
            )

            # Read the data
            data = src.read()

            # Write to the new file
            with rasterio.open(str(self.tif_path), "w", **kwargs) as dst:
                dst.write(data)

        print("Saving", self.tif_path)

    def __str__(self):
        "Print out some infos about yourself."
        s = self.edr.__str__()
        try:
            s += f"Shape: {self.edr_da.shape}"
        except FileNotFoundError:
            s += "Not downloaded yet."
        return s

    def __repr__(self):
        return self.__str__()

In [36]:
pid = "N05_064260_1638_XI_16S351W"

In [37]:
ctx = CTX(pid)

In [39]:
ctx.download()

N05_064260_1638_XI_16S351W.IMG:   0%|          | 0/77665216 [00:00<?, ?it/s]

In [41]:
str(ctx.map_path)

'/Users/maye/planetarypy_data/missions/mro/ctx/edr/N05_064260_1638_XI_16S351W.lev2.cub'

In [42]:
ctx.proc_folder / ctx.map_name.with_suffix(".tif")

Path('/Users/maye/planetarypy_data/missions/mro/ctx/edr/N05_064260_1638_XI_16S351W.lev2.tif')

In [43]:
ctx.cal_da

RasterioIOError: /Users/maye/planetarypy_data/missions/mro/ctx/edr/N05_064260_1638_XI_16S351W.cal.cub: No such file or directory

In [44]:
ctx.preproc_cal_path

Path('N05_064260_1638_XI_16S351W')

Based on storage options `with_pid_folder` and `with_volume`, we calculate the `proc_folder` for self-processed data:

In [45]:
show_doc(CTX.proc_folder)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L199){target="_blank" style="float:right; font-size:smaller"}

### CTX.proc_folder

>      CTX.proc_folder ()

*the folder for foreign processed data, like pre-processed calibrated data, e.g.*

In [46]:
ctx.proc_folder

Path('/Users/maye/planetarypy_data/missions/mro/ctx/edr')

These can be changed at object creation:

In [47]:
CTX(pid, with_volume=True, with_pid_folder=True).source_folder

Path('/Users/maye/planetarypy_data/missions/mro/ctx/edr/mrox_3629')

In [48]:
show_doc(CTX.cal_path)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L224){target="_blank" style="float:right; font-size:smaller"}

### CTX.cal_path

>      CTX.cal_path ()

*Path to calibrated cube file. Also destriped files get this name.*

In [49]:
ctx.cal_path

Path('/Users/maye/planetarypy_data/missions/mro/ctx/edr/N05_064260_1638_XI_16S351W.cal.cub')

In [50]:
show_doc(CTX.calib_pipeline)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L285){target="_blank" style="float:right; font-size:smaller"}

### CTX.calib_pipeline

>      CTX.calib_pipeline (overwrite=False)

*Execute the whole ISIS pipeline for CTX EDR data.*

In [51]:
ctx.proc_folder

Path('/Users/maye/planetarypy_data/missions/mro/ctx/edr')

In [52]:
ctx.isis_import()

In [53]:
ctx.spice_init(web="yes")

In [54]:
ctx.calibrate()

In [55]:
ctx.destripe()

In [ ]:
# not executing always, as it takes lot of time
# ctx.map_project()

In [ ]:
ctx.map_path

In [56]:
ctx.calib_pipeline()

In [57]:
show_doc(CTX.plot_edr)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L346){target="_blank" style="float:right; font-size:smaller"}

### CTX.plot_edr

>      CTX.plot_edr ()

*Plot EDR xarray using hvplot.*

In [59]:
ctx.plot_edr()

BokehModel(combine_events=True, render_bundle={'docs_json': {'c2f400e0-63d4-40c7-8804-f0c56895fd91': {'version…

:::{.callout-note}
Note the different shape of EDR data and calibrated data. A few SAMPLES are being used for calibration.
:::

In [60]:
ctx.edr_shape

(15360, 5056)

In [61]:
ctx.cal_shape

(15360, 5000)

In [62]:
ctx.plot_calibrated()

BokehModel(combine_events=True, render_bundle={'docs_json': {'4b95e8d7-731b-4e9e-b1f9-3721ae93f71d': {'version…

## CTXCollection -

In [63]:
# | export


class CTXCollection:
    """Class with several helpful methods to work with a set of CTX images.

    We identify the images via a list of product_ids.
    Several methods manipulate this list based on the requested constraint.
    """

    @classmethod
    def by_volume(cls, vol_id, **kwargs):
        """Create a CTXCollection from the PDS volume number."""
        if not str(vol_id).startswith("MROX_"):
            vol_id = "MROX_" + str(vol_id)
        query = f"VOLUME_ID=='{vol_id}'"
        edrindex = get_edr_index()
        return cls(edrindex.query(query).PRODUCT_ID.values, edrindex=edrindex, **kwargs)

    @classmethod
    def by_month(cls, month_letters, nth_volume=None, **kwargs):
        """Create a CTXCollection based on the first 3 letters of the product_id (a.k.a. "month")"""
        edrindex = get_edr_index()
        df = edrindex[edrindex.PRODUCT_ID.str.startswith(month_letters)]
        obj = cls(df.PRODUCT_ID.values, **kwargs)
        if nth_volume is not None:
            return cls.by_volume(
                obj.volumes_in_pids[nth_volume], edrindex=edrindex, **kwargs
            )
        else:
            return obj

    @classmethod
    def volume_from_pid(cls, pid, **kwargs):
        """Get a CTXCollection of the volume for a given image (product_id)."""
        edrindex = get_edr_index()
        vol = edrindex.query(f"PRODUCT_ID=='{pid}'").VOLUME_ID.iat[0]
        return cls.by_volume(vol, **kwargs)

    def __init__(
        self, product_ids, full_width=False, filter_error=False, edrindex=None
    ):
        self.product_ids = product_ids
        self.full_width = full_width  # i.e. LINE_SAMPLES==5056
        self.filter_error = filter_error
        self.edrindex = get_edr_index() if edrindex is None else edrindex

    @property
    def pids(self):
        "Alias on product_id"
        return self.product_ids

    @property
    def product_ids(self):
        new_pids = self._product_ids
        ind = self.edrindex[self.edrindex.PRODUCT_ID.isin(new_pids)]
        queries = []
        if self.full_width:
            queries.append("LINE_SAMPLES == 5056")
            # new_pids = [pid for pid in new_pids if CTX(pid).meta.line_samples == 5056]
        if self.filter_error:
            queries.append("DATA_QUALITY_DESC != 'ERROR'")
            # new_pids = [pid for pid in new_pids if CTX(pid).data_quality != 'ERROR']
        if queries:
            return ind.query(" and ".join(queries)).PRODUCT_ID.values
        else:
            return ind.PRODUCT_ID.values

    @product_ids.setter
    def product_ids(self, val):
        self._product_ids = val

    def get_urls(self):
        """Get URLs for list of product_ids.

        Returns
        -------
        List[yarl.URL]
            List of URL objects with the respective PDS URL for download.
        """
        urls = []
        for p_id in self.product_ids:
            ctx = CTXEDR(p_id)
            urls.append(ctx.url)
        self.urls = urls
        return urls

    def _do_download(self, args):
        pid, overwrite = args
        ctx = CTX(pid)
        ctx.download(overwrite=overwrite)

    def download_collection(self, overwrite=False):
        "download the images in parallel using tqdm wrapper around concurrent.future"
        print("Downloading collection...")
        args = zip(self.product_ids, repeat(overwrite))
        r = process_map(self._do_download, args, max_workers=6)

    def _do_calib(self, args):
        pid, overwrite = args
        ctx = CTX(pid)
        ctx.calib_pipeline(overwrite=overwrite)

    def calibrate_collection(self, overwrite=False):
        "Calibrate all images in collection using tqdm wrapper around concurrent.future"
        print("Launching parallel calibration...")
        args = zip(self.product_ids, repeat(overwrite))
        process_map(self._do_calib, args, max_workers=6)

    def edr_exist_check(self):
        "Check if all source_paths exists, i.e. all EDR images are available."
        return [(p_id, CTX(p_id).source_path.exists()) for p_id in self.product_ids]

    def calib_exist_check(self):
        "Check if all cal_paths exist. (i.e. all calibrated ISIS cubes are available."
        return [(p_id, CTX(p_id).cal_path.exists()) for p_id in self.product_ids]

    def only_full_width(self):
        "Constrain the list of product_ids to those that have full width (i.e. line_samples == 5056)"

    def get_ctx_n(self, n):
        "Get CTX object for n-th product_id"
        return CTX(self.product_ids[n])

    def get_pid_n(self, n):
        "Get pid for n-th entry in product_ids."
        return self.product_ids[n]

    @property
    def n_items(self):
        "Return length of product_ids list."
        return len(self.pids)

    @property
    def meta(self):
        "Return the index file filtered for the given product_ids."
        return self.edrindex[self.edrindex.PRODUCT_ID.isin(self.pids)]

    @property
    def image_times(self):
        "Return the image observation times."
        return self.meta.IMAGE_TIME

    def get_corrupted(self):
        "Return the product_ids where the PDS index file has an 'ERROR' flag for the `DATA_QUALITY_DESC` field."
        return [pid for pid in self.pids if CTX(pid).data_quality == "ERROR"]

    def filter_error(self):
        "Filter the product_ids for the error flag from the PDS index."
        self.product_ids = [
            pid for pid in self.pids if CTX(pid).data_quality != "ERROR"
        ]

    @property
    def volumes_in_pids(self):
        return self.edrindex[
            self.edrindex.PRODUCT_ID.isin(self.product_ids)
        ].VOLUME_ID.unique()

    @property
    def count_per_volume(self):
        g = self.edrindex.groupby("VOLUME_ID")
        return g.size()[self.volumes_in_pids]

    def sample(self, n):
        "Return random sample of product_ids, size `n`."
        return list(pd.Series(self.product_ids).sample(n))

    def __str__(self):
        s = f"# of product IDs: {self.n_items}\n"
        s += "Volumes contained in list of product_ids:\n"
        s += f"{self.volumes_in_pids}\n"
        return s

    def __repr__(self):
        return self.__str__()

The `CTXCollection` class offers a few class methods for a wider range of finding CTX product_ids from the index file:

In [64]:
show_doc(CTXCollection.by_volume)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L387){target="_blank" style="float:right; font-size:smaller"}

### CTXCollection.by_volume

>      CTXCollection.by_volume (vol_id, **kwargs)

*Create a CTXCollection from the PDS volume number.*

In [65]:
CTXCollection.by_volume(4114).n_items

30

In [66]:
CTXCollection.by_volume(4114, full_width=True).n_items

19

In [67]:
CTXCollection.by_volume(4114, full_width=True, filter_error=True).n_items

19

In [68]:
CTXCollection.by_volume(4114, full_width=False, filter_error=True).n_items

30

In [69]:
show_doc(CTXCollection.by_month)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L396){target="_blank" style="float:right; font-size:smaller"}

### CTXCollection.by_month

>      CTXCollection.by_month (month_letters, nth_volume=None, **kwargs)

*Create a CTXCollection based on the first 3 letters of the product_id (a.k.a. "month")*

In [70]:
CTXCollection.by_month("J18", filter_error=True, full_width=True).n_items

304

In [71]:
show_doc(CTXCollection.volume_from_pid)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L409){target="_blank" style="float:right; font-size:smaller"}

### CTXCollection.volume_from_pid

>      CTXCollection.volume_from_pid (pid, **kwargs)

*Get a CTXCollection of the volume for a given image (product_id).*

We define an example list of `product_id`s:

In [72]:
ids = get_edr_index().sample(3, random_state=41).PRODUCT_ID
ids

136286    U12_075730_1229_XN_57S226W
22759     B11_013783_1919_XN_11N021W
51718     D05_029090_0936_XN_86S261W
Name: PRODUCT_ID, dtype: string

In [73]:
CTXCollection.volume_from_pid(ids.values[0]).n_items  # getting the whole volume here

20

In [74]:
coll = CTXCollection(ids)

In [75]:
coll.edr_exist_check()

[('B11_013783_1919_XN_11N021W', False),
 ('D05_029090_0936_XN_86S261W', False),
 ('U12_075730_1229_XN_57S226W', False)]

In [76]:
coll.get_urls()

[URL('https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_0845/data/B11_013783_1919_XN_11N021W.IMG'),
 URL('https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_1752/data/D05_029090_0936_XN_86S261W.IMG'),
 URL('https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_4376/data/U12_075730_1229_XN_57S226W.IMG')]

The next command launches a parallel download:

In [77]:
coll.download_collection(overwrite=False)

0it [00:00, ?it/s]

This is performing the ISIS import and calibration in parallel:

In [ ]:
coll.calibrate_collection()

In [ ]:
coll.calib_exist_check()

[('G02_018931_1907_XI_10N166W', False),
 ('G16_024548_2195_XI_39N161W', False),
 ('G20_026104_2617_XN_81N181W', False)]

In [ ]:
coll = CTXCollection.by_volume(4114)

In [ ]:
coll.product_ids

<StringArray>
['N20_069979_1676_XI_12S177W', 'N20_069980_1676_XI_12S205W',
 'N20_069981_1919_XI_11N234W', 'N20_069982_1380_XI_42S255W',
 'N20_069982_1820_XI_02N261W', 'N20_069982_2287_XN_48N269W',
 'N20_069983_1442_XI_35S283W', 'N20_069984_1686_XI_11S313W',
 'N20_069984_2097_XI_29N319W', 'N20_069985_2064_XI_26N345W',
 'N20_069986_2025_XI_22N012W', 'N20_069987_2243_XN_44N042W',
 'N20_069991_1451_XI_34S142W', 'N20_069991_1940_XI_14N149W',
 'N20_069992_1761_XI_03S173W', 'N20_069993_1724_XI_07S200W',
 'N20_069994_1753_XI_04S227W', 'N20_069995_1633_XI_16S254W',
 'N20_069995_2028_XI_22N258W', 'N20_069996_2085_XN_28N285W',
 'N20_069997_1479_XI_32S306W', 'N20_069999_1558_XI_24S003W',
 'N20_070004_1931_XI_13N144W', 'N20_070006_1430_XI_37S191W',
 'N20_070007_1793_XI_00S223W', 'N20_070009_2018_XN_21N282W',
 'N20_070010_1466_XI_33S301W', 'N20_070011_1507_XI_29S328W',
 'N20_070011_2252_XI_45N338W', 'N20_070012_1824_XN_02N001W']
Length: 30, dtype: string

In [ ]:
show_doc(CTXCollection.get_corrupted)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L506){target="_blank" style="float:right; font-size:smaller"}

### CTXCollection.get_corrupted

>      CTXCollection.get_corrupted ()

Return the product_ids where the PDS index file has an 'ERROR' flag for the `DATA_QUALITY_DESC` field.

In [ ]:
coll.get_corrupted()

['N20_069991_1451_XI_34S142W',
 'N20_069992_1761_XI_03S173W',
 'N20_069993_1724_XI_07S200W',
 'N20_069994_1753_XI_04S227W',
 'N20_069997_1479_XI_32S306W',
 'N20_070010_1466_XI_33S301W',
 'N20_070012_1824_XN_02N001W']

In [ ]:
show_doc(CTXCollection.n_items)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L492){target="_blank" style="float:right; font-size:smaller"}

### CTXCollection.n_items

>      CTXCollection.n_items ()

Return length of product_ids list.

In [ ]:
coll.n_items

30

In [ ]:
show_doc(CTXCollection.sample)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L523){target="_blank" style="float:right; font-size:smaller"}

### CTXCollection.sample

>      CTXCollection.sample (n)

Return random sample of product_ids, size `n`.

In [ ]:
coll.sample(4)

['N20_070006_1430_XI_37S191W',
 'N20_069999_1558_XI_24S003W',
 'N20_069986_2025_XI_22N012W',
 'N20_069983_1442_XI_35S283W']

In [ ]:
show_doc(CTXCollection.meta)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L497){target="_blank" style="float:right; font-size:smaller"}

### CTXCollection.meta

>      CTXCollection.meta ()

Return the index file filtered for the given product_ids.

In [ ]:
coll.meta.head()

,VOLUME_ID,FILE_SPECIFICATION_NAME,ORIGINAL_PRODUCT_ID,PRODUCT_ID,IMAGE_TIME,INSTRUMENT_ID,INSTRUMENT_MODE_ID,LINE_SAMPLES,LINES,SPATIAL_SUMMING,SCALED_PIXEL_WIDTH,PIXEL_ASPECT_RATIO,EMISSION_ANGLE,INCIDENCE_ANGLE,PHASE_ANGLE,CENTER_LONGITUDE,CENTER_LATITUDE,UPPER_LEFT_LONGITUDE,UPPER_LEFT_LATITUDE,UPPER_RIGHT_LONGITUDE,UPPER_RIGHT_LATITUDE,LOWER_LEFT_LONGITUDE,LOWER_LEFT_LATITUDE,LOWER_RIGHT_LONGITUDE,LOWER_RIGHT_LATITUDE,MISSION_PHASE_NAME,TARGET_NAME,SPACECRAFT_CLOCK_START_COUNT,FOCAL_PLANE_TEMPERATURE,LINE_EXPOSURE_DURATION,OFFSET_MODE_ID,SAMPLE_FIRST_PIXEL,SCALED_IMAGE_WIDTH,SCALED_IMAGE_HEIGHT,SPACECRAFT_ALTITUDE,TARGET_CENTER_DISTANCE,SLANT_DISTANCE,USAGE_NOTE,NORTH_AZIMUTH,SUB_SOLAR_AZIMUTH,SUB_SOLAR_LONGITUDE,SUB_SOLAR_LATITUDE,SUB_SPACECRAFT_LONGITUDE,SUB_SPACECRAFT_LATITUDE,SOLAR_DISTANCE,SOLAR_LONGITUDE,LOCAL_TIME,IMAGE_SKEW_ANGLE,RATIONALE_DESC,DATA_QUALITY_DESC,ORBIT_NUMBER,short_pid,month_col
127546,MROX_4114,DATA/N20_069979_1676_XI_12S177W.IMG,4A_04_1165000100,N20_069979_1676_XI_12S177W,2021-07-01 02:22:53.651,CTX,ITL,5056,43008,1,5.3,1.14,5.3,65.18,61.08,177.4,-12.53,177.37,-14.74,176.91,-14.69,177.89,-10.37,177.43,-10.32,ESP,MARS,1309573428:228,291.3,1.877,196/188/183,0,26.53,260.74,262.71,3657.97,263.76,N,276.79,219.82,233.3,23.12,177.78,-12.56,249116269.6,65.97,15.72,90.2,Northern Terra Sirenum,OK,69979,N20_069979_1676,N20
127547,MROX_4114,DATA/N20_069980_1676_XI_12S205W.IMG,4A_04_1165000200,N20_069980_1676_XI_12S205W,2021-07-01 04:15:32.600,CTX,ITL,5056,18432,1,5.26,1.15,2.48,64.99,63.03,204.93,-12.5,205.04,-13.46,204.59,-13.41,205.26,-11.59,204.81,-11.53,ESP,MARS,1309580187:215,290.9,1.877,196/188/183,0,26.34,111.73,262.54,3657.8,262.77,N,276.75,219.87,260.62,23.13,205.11,-12.51,249117636.2,66.0,15.71,90.1,Valleys in Terra Cimmeria,OK,69980,N20_069980_1676,N20
127548,MROX_4114,DATA/N20_069981_1919_XI_11N234W.IMG,4A_04_1165000300,N20_069981_1919_XI_11N234W,2021-07-01 06:14:46.674,CTX,ITL,5056,52224,1,5.62,1.07,8.13,53.43,46.2,234.64,11.91,234.58,9.25,234.1,9.3,235.2,14.52,234.71,14.57,ESP,MARS,1309587341:234,291.1,1.877,196/188/183,0,28.12,314.36,275.31,3670.67,277.89,N,276.65,206.74,289.76,23.14,235.27,11.85,249118514.0,66.04,15.66,90.0,Nepenthes Planum region,OK,69981,N20_069981_1919,N20
127549,MROX_4114,DATA/N20_069982_1380_XI_42S255W.IMG,4A_04_1165000400,N20_069982_1380_XI_42S255W,2021-07-01 07:50:56.614,CTX,ITL,2528,7168,2,10.25,1.19,4.87,83.98,80.5,255.35,-42.07,255.52,-42.83,254.94,-42.77,255.75,-41.37,255.18,-41.32,ESP,MARS,1309593111:219,291.2,1.884,196/188/183,0,25.65,87.09,254.99,3642.26,255.85,N,276.84,225.48,313.02,23.14,255.8,-42.1,249121323.6,66.07,15.84,90.1,Apron in the Hellas Montes region,OK,69982,N20_069982_1380,N20
127550,MROX_4114,DATA/N20_069982_1820_XI_02N261W.IMG,4A_04_1165000500,N20_069982_1820_XI_02N261W,2021-07-01 08:03:55.615,CTX,ITL,5056,52224,1,5.42,1.11,5.11,57.47,53.17,261.02,2.02,260.94,-0.66,260.49,-0.6,261.57,4.63,261.11,4.69,ESP,MARS,1309593890:219,291.0,1.877,196/188/183,0,27.13,315.79,268.72,3664.89,269.71,N,276.84,213.19,316.33,23.14,261.4,1.98,249120029.2,66.07,15.67,90.1,Tyrrhena Terra,OK,69982,N20_069982_1820,N20


In [ ]:
show_doc(CTXCollection.image_times)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L502){target="_blank" style="float:right; font-size:smaller"}

### CTXCollection.image_times

>      CTXCollection.image_times ()

Return the image observation times.

In [ ]:
coll.image_times

127546   2021-07-01 02:22:53.651
127547   2021-07-01 04:15:32.600
127548   2021-07-01 06:14:46.674
127549   2021-07-01 07:50:56.614
127550   2021-07-01 08:03:55.615
127551   2021-07-01 08:19:18.963
127552   2021-07-01 09:44:54.631
127553   2021-07-01 11:44:29.607
127554   2021-07-01 11:57:15.646
127555   2021-07-01 13:48:19.595
127556   2021-07-01 15:39:04.638
127557   2021-07-01 17:38:53.212
127558   2021-07-02 00:42:16.641
127559   2021-07-02 00:57:36.688
127560   2021-07-02 02:44:15.621
127561   2021-07-02 04:35:30.668
127562   2021-07-02 06:28:38.621
127563   2021-07-02 08:16:50.660
127564   2021-07-02 08:29:27.601
127565   2021-07-02 10:23:41.558
127566   2021-07-02 11:56:54.648
127567   2021-07-02 15:43:37.659
127568   2021-07-03 01:16:03.651
127569   2021-07-03 04:45:28.607
127570   2021-07-03 06:48:35.596
127571   2021-07-03 10:40:22.119
127572   2021-07-03 12:14:51.615
127573   2021-07-03 14:08:48.673
127574   2021-07-03 14:31:39.582
127575   2021-07-03 16:10:56.454
Name: IMAG

Also cool: `pandas` can do time-average:

In [ ]:
coll.image_times.mean()

Timestamp('2021-07-02 05:45:11.936933376')

In [ ]:
show_doc(CTXCollection.get_ctx_n)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L483){target="_blank" style="float:right; font-size:smaller"}

### CTXCollection.get_ctx_n

>      CTXCollection.get_ctx_n (n)

Get CTX object for n-th product_id

In [ ]:
coll.get_ctx_n(2)

PRODUCT_ID: N20_069981_1919_XI_11N234W
URL: https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_4114/data/N20_069981_1919_XI_11N234W.IMG
source_path: /remote/trove/geo/planet/Mars/CTX/pds/mrox_4114/N20_069981_1919_XI_11N234W.IMG
Shape: (52224, 5056)

In [ ]:
show_doc(CTXCollection.get_pid_n)

---

[source](https://github.com/michaelaye/nbplanetary/blob/master/planetarypy/ctx.py#L487){target="_blank" style="float:right; font-size:smaller"}

### CTXCollection.get_pid_n

>      CTXCollection.get_pid_n (n)

Get pid for n-th entry in product_ids.

In [ ]:
coll.get_pid_n(2)

'N20_069981_1919_XI_11N234W'

In [ ]:
coll = CTXCollection.by_month("N21")

In [ ]:
len(coll.product_ids)

342

In [ ]:
coll

# of product IDs: 342
Volumes contained in list of product_ids:
<StringArray>
['MROX_4126', 'MROX_4127', 'MROX_4128', 'MROX_4129', 'MROX_4130', 'MROX_4131',
 'MROX_4132', 'MROX_4133']
Length: 8, dtype: string

## Command line interfaces

In [ ]:
# | export
@call_parse
def ctx_calib(
    pid: str,  # CTX product_id
    source: str = "",  # path to where EDRs are stored if not from plpy
    proc_root: str = "",  # path to where processed data is to be stored
    overwrite: bool = False,  # overwrite processed data
):
    ctx = CTX(pid, source_dir=source, proc_root=proc_root)
    ctx.calib_pipeline(overwrite=overwrite)
    print("Produced\n", ctx.cal_path)

In [ ]:
ctx_calib(pid, overwrite=True)

In [ ]:
from nbdev import nbdev_export

nbdev_export()